In [1]:
%cd nanoMoD

/teamspace/studios/this_studio/nanoMoD


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [11]:
import torch
import wandb
from hydra import compose, initialize
from pprint import pprint

from nanomod import utils
from nanomod.model import DnasSearchModel, GPT
from nanomod.dataset import get_dataloaders
from nanomod.configuration import set_config_store

In [8]:
set_config_store()
with initialize(version_base=None, config_path="config"):
    cfg = compose(config_name="config_search", overrides=["data=distil", "train=dnas/distil", "dnas=distil"])

In [15]:
pprint(dict(cfg.data))

{'batch_size': 64,
 'num_tokens': 5000000,
 'num_workers': 4,
 'pin_memory': True,
 'seed': 42,
 'seq_len': 256,
 'train_dataset': 'random'}


In [14]:
pprint(dict(cfg.train))

{'distillation_loss': 'mse',
 'epochs': 3,
 'eval_iterations': 100,
 'grad_clip_alphas': None,
 'grad_clip_model': None,
 'log_interval': 50,
 'lr_alphas': 0.006,
 'lr_model': 0.0006,
 'max_steps': 1500,
 'train_router_steps': 0.5,
 'use_distillation': True,
 'use_fp16': True,
 'use_wandb': True,
 'watch_mode': 'gradients',
 'watch_model': False}


In [16]:
pprint(dict(cfg.dnas))

{'a': 0.6,
 'all_trainable': False,
 'b': 2.0,
 'capacity_ratio_search_space': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
 'compute_mode': 'sqrt',
 'fix_first_last': True,
 'gumbel_temperature': 6.0,
 'hard_sampling': True,
 'share_router_weights': False}


In [17]:
state_dict, model_config = utils.load_checkpoint(use_wandb=False)

In [18]:
base_model = GPT(model_config)
base_model.load_state_dict(state_dict)

number of parameters: 29.96M


<All keys matched successfully>

In [26]:
model = DnasSearchModel(base_model, cfg.dnas)
model.freeze()

In [17]:
train_loader, val_loader = get_dataloaders(cfg.data)

In [18]:
inputs, targets = next(iter(train_loader))

In [22]:
with torch.no_grad():
    logits, loss = base_model(inputs, targets)

In [27]:
with torch.no_grad():
    logits, loss = model(inputs, targets)

In [28]:
loss

tensor(12.4444)

In [23]:
loss

tensor(5.2774)